In [1]:
# 导入核心库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')  # 忽略无关警告

In [2]:

# 设置中文字体（用于论文图表）
plt.rcParams['font.family']=' Times New Roman, SimSun' #兼容英文和中文
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

In [3]:

# --------------------------
# 步骤1：加载数据并查看基础信息
# --------------------------
# 读取CSV文件（请确保文件路径与您的环境一致）
df = pd.read_csv('sort_summary.csv')

# 1.1 查看数据前5行与列名（确认指标列）
print("=== 数据前5行 ===")
print(df.head())

=== 数据前5行 ===
   Unnamed: 0       mota      motp      idf1  num_switches  num_objects  \
0           0  -3.895317  0.201500  0.203647           191         1089   
1           1 -10.202295  0.235513  0.115099           632         4271   
2           2  -1.761678  0.235548  0.355489           134         2098   
3           3  -0.789779  0.171093  0.525261            27          861   
4           4  -2.729125  0.232437  0.337063           309         2012   

   precision    recall  
0   0.164708  0.913682  
1   0.070837  0.829782  
2   0.227593  0.709247  
3   0.272474  0.454123  
4   0.168670  0.655567  


In [4]:

print("\n=== 数据列名 ===")
print(df.columns.tolist())  # 确认是否包含：mota, motp, idf1, num_switches, num_objects, precision, recall


=== 数据列名 ===
['Unnamed: 0', 'mota', 'motp', 'idf1', 'num_switches', 'num_objects', 'precision', 'recall']


In [5]:

# 1.2 筛选核心指标列（排除无用列，如序列名称列，根据实际列名调整）
# 假设序列名称列为"sequence_name"，核心指标列为以下7项
core_cols = ['mota', 'motp', 'idf1', 'num_switches', 'num_objects', 'precision', 'recall']
df_core = df[core_cols].copy()  # 核心指标数据框

In [6]:

# 1.3 数据完整性检查（缺失值、异常值）
print("\n=== 数据完整性检查 ===")
print(f"缺失值统计：\n{df_core.isnull().sum()}")  # 检查缺失值


=== 数据完整性检查 ===
缺失值统计：
mota            0
motp            0
idf1            0
num_switches    0
num_objects     0
precision       0
recall          0
dtype: int64


In [7]:

# 异常值判断（基于指标物理意义）
abnormal_log = []
# - MOTA：通常在[-100, 100]之间（百分比），超出则标记
abnormal_mota = df_core[(df_core['mota'] < -100) | (df_core['mota'] > 100)].index.tolist()
if abnormal_mota:
    abnormal_log.append(f"MOTA异常（超出[-100,100]）：序列索引{abnormal_mota}")



In [8]:
# - MOTP：位置偏差，通常在[0, 1]之间（越小越好），>1标记
abnormal_motp = df_core[df_core['motp'] > 1].index.tolist()
if abnormal_motp:
    abnormal_log.append(f"MOTP异常（>1）：序列索引{abnormal_motp}")



In [9]:
# - Precision/Recall/IDF1：百分比，超出[0, 100]标记
for col in ['idf1', 'precision', 'recall']:
    abnormal = df_core[(df_core[col] < 0) | (df_core[col] > 100)].index.tolist()
    if abnormal:
        abnormal_log.append(f"{col}异常（超出[0,100]）：序列索引{abnormal}")



In [10]:
# - 计数类指标（num_switches/num_objects）：不能为负
for col in ['num_switches', 'num_objects']:
    abnormal = df_core[df_core[col] < 0].index.tolist()
    if abnormal:
        abnormal_log.append(f"{col}异常（<0）：序列索引{abnormal}")

In [11]:

# 打印异常值日志（无异常则提示正常）
if abnormal_log:
    for log in abnormal_log:
        print(log)
else:
    print("无异常值，数据符合指标物理意义")

无异常值，数据符合指标物理意义


In [12]:

# 1.4 数据标准化（针对计数类指标，消除量纲影响）
# 需要标准化的指标：num_switches（ID切换次数）、num_objects（目标总数）
scaler = StandardScaler()
df_core[['num_switches_std', 'num_objects_std']] = scaler.fit_transform(
    df_core[['num_switches', 'num_objects']]
)

# 标准化后的核心指标列（用于后续Pearson相关分析）
std_core_cols = ['mota', 'motp', 'idf1', 'num_switches_std', 'num_objects_std', 'precision', 'recall']
df_std = df_core[std_core_cols].copy()

print("\n=== 数据预处理完成 ===")
print(f"标准化后数据形状：{df_std.shape}")
print(f"标准化后数据前3行：\n{df_std.head(3)}")


=== 数据预处理完成 ===
标准化后数据形状：(21, 7)
标准化后数据前3行：
        mota      motp      idf1  num_switches_std  num_objects_std  \
0  -3.895317  0.201500  0.203647         -0.456531        -0.768526   
1 -10.202295  0.235513  0.115099          0.593104         0.441750   
2  -1.761678  0.235548  0.355489         -0.592198        -0.384752   

   precision    recall  
0   0.164708  0.913682  
1   0.070837  0.829782  
2   0.227593  0.709247  


In [13]:
# --------------------------
# 步骤2：全指标整体相关性分析
# --------------------------
# 2.1 计算两种相关系数矩阵（Pearson：线性相关；Spearman：秩相关，适用于非正态数据）
# Pearson相关系数（使用标准化后的数据）
pearson_corr, pearson_p = {}, {}
for col1 in std_core_cols:
    pearson_corr[col1] = []
    pearson_p[col1] = []
    for col2 in std_core_cols:
        corr, p_val = pearsonr(df_std[col1], df_std[col2])
        pearson_corr[col1].append(round(corr, 3))
        pearson_p[col1].append(round(p_val, 4))

pearson_corr_df = pd.DataFrame(pearson_corr, index=std_core_cols)
pearson_p_df = pd.DataFrame(pearson_p, index=std_core_cols)

In [14]:

# Spearman相关系数（使用原始数据，无需标准化）
spearman_corr, spearman_p = {}, {}
for col1 in core_cols:
    spearman_corr[col1] = []
    spearman_p[col1] = []
    for col2 in core_cols:
        corr, p_val = spearmanr(df_core[col1], df_core[col2])
        spearman_corr[col1].append(round(corr, 3))
        spearman_p[col1].append(round(p_val, 4))

spearman_corr_df = pd.DataFrame(spearman_corr, index=core_cols)
spearman_p_df = pd.DataFrame(spearman_p, index=core_cols)

In [15]:

# 打印相关系数矩阵（论文中可直接引用表格）
print("=== Pearson相关系数矩阵（标准化数据） ===")
print(pearson_corr_df)
print("\n=== Pearson相关性P值矩阵（<0.05为显著，<0.01为极显著） ===")
print(pearson_p_df)

print("\n=== Spearman相关系数矩阵（原始数据） ===")
print(spearman_corr_df)
print("\n=== Spearman相关性P值矩阵 ===")
print(spearman_p_df)

=== Pearson相关系数矩阵（标准化数据） ===
                   mota   motp   idf1  num_switches_std  num_objects_std  \
mota              1.000 -0.417  0.732            -0.814           -0.796   
motp             -0.417  1.000 -0.650             0.491            0.434   
idf1              0.732 -0.650  1.000            -0.613           -0.582   
num_switches_std -0.814  0.491 -0.613             1.000            0.954   
num_objects_std  -0.796  0.434 -0.582             0.954            1.000   
precision         0.736 -0.557  0.947            -0.594           -0.555   
recall           -0.517  0.431 -0.629             0.604            0.613   

                  precision  recall  
mota                  0.736  -0.517  
motp                 -0.557   0.431  
idf1                  0.947  -0.629  
num_switches_std     -0.594   0.604  
num_objects_std      -0.555   0.613  
precision             1.000  -0.447  
recall               -0.447   1.000  

=== Pearson相关性P值矩阵（<0.05为显著，<0.01为极显著） ===
              

In [16]:

# 2.2 筛选强相关对（|r|≥0.6 且 P<0.01）
print("\n=== 强相关对筛选结果（|r|≥0.6 且 P<0.01） ===")
strong_corr = []
# 遍历Pearson结果（以Pearson为例，可替换为Spearman）
for i, col1 in enumerate(std_core_cols):
    for j, col2 in enumerate(std_core_cols):
        if i < j:  # 避免重复（只看上三角）
            corr = pearson_corr_df.loc[col1, col2]
            p_val = pearson_p_df.loc[col1, col2]
            if abs(corr) >= 0.6 and p_val < 0.01:
                strong_corr.append({
                    '指标1': col1,
                    '指标2': col2,
                    'Pearson相关系数': corr,
                    'P值': p_val,
                    '相关性强度': '强' + ('正相关' if corr > 0 else '负相关')
                })

# 转换为DataFrame并打印
strong_corr_df = pd.DataFrame(strong_corr)
if not strong_corr_df.empty:
    print(strong_corr_df)
else:
    print("未找到满足条件的强相关对（可降低阈值至|r|≥0.5）")


=== 强相关对筛选结果（|r|≥0.6 且 P<0.01） ===
                 指标1               指标2  Pearson相关系数      P值 相关性强度
0               mota              idf1        0.732  0.0002  强正相关
1               mota  num_switches_std       -0.814  0.0000  强负相关
2               mota   num_objects_std       -0.796  0.0000  强负相关
3               mota         precision        0.736  0.0001  强正相关
4               motp              idf1       -0.650  0.0014  强负相关
5               idf1  num_switches_std       -0.613  0.0032  强负相关
6               idf1         precision        0.947  0.0000  强正相关
7               idf1            recall       -0.629  0.0023  强负相关
8   num_switches_std   num_objects_std        0.954  0.0000  强正相关
9   num_switches_std            recall        0.604  0.0037  强正相关
10   num_objects_std            recall        0.613  0.0031  强正相关


In [17]:

# 2.3 绘制相关性热力图（论文用图，高分辨率）
# 替换标准化列名为易读名称（如num_switches_std→ID切换次数（标准化））
label_map = {
    'mota': 'MOTA(%)',
    'motp': 'MOTP',
    'idf1': 'IDF1(%)',
    'num_switches_std': 'ID切换次数（标准化）',
    'num_objects_std': '目标总数（标准化）',
    'precision': 'Precision(%)',
    'recall': 'Recall(%)'
}
plot_labels = [label_map[col] for col in std_core_cols]

# 创建画布
fig, ax = plt.subplots(figsize=(12, 10))  # 图大小适配论文
im = ax.imshow(pearson_corr_df.values, cmap='RdBu_r', vmin=-1, vmax=1)  # 红蓝色谱（-1负相关，1正相关）

# 添加颜色条（标注相关系数范围）
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel('Pearson相关系数', rotation=-90, va="bottom", fontsize=11)

# 设置坐标轴标签
ax.set_xticks(np.arange(len(plot_labels)))
ax.set_yticks(np.arange(len(plot_labels)))
ax.set_xticklabels(plot_labels, rotation=45, ha="right", rotation_mode="anchor", fontsize=10)
ax.set_yticklabels(plot_labels, fontsize=10)

# 在热力图中添加数值+显著性标记
for i in range(len(plot_labels)):
    for j in range(len(plot_labels)):
        corr_val = pearson_corr_df.iloc[i, j]
        p_val = pearson_p_df.iloc[i, j]
        # 显著性标记：***p<0.001, **p<0.01, *p<0.05, ns不显著
        if p_val < 0.001:
            sig_mark = '***'
        elif p_val < 0.01:
            sig_mark = '**'
        elif p_val < 0.05:
            sig_mark = '*'
        else:
            sig_mark = 'ns'
        # 数值颜色：绝对值>0.6用白色（突出强相关），否则黑色
        text_color = 'white' if abs(corr_val) >= 0.6 else 'black'
        # 添加文本（相关系数+显著性）
        ax.text(j, i, f'{corr_val:.3f}\n{sig_mark}',
                ha="center", va="center", color=text_color, fontsize=9, fontweight='bold')

# 设置标题（适配论文格式）
ax.set_title('多目标跟踪指标Pearson相关性热力图\n（*p<0.05, **p<0.01, ***p<0.001, ns不显著）',
             fontsize=13, pad=20)
fig.tight_layout()  # 自动调整布局，避免标签截断

# 保存图片（高分辨率，适合论文插入）
plt.savefig('overall_correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()

print("\n=== 全指标相关性分析完成 ===")
print(f"热力图已保存至：overall_correlation_heatmap.png")
print(f"强相关对结果已保存至：strong_corr_df（可导出为CSV）")


=== 全指标相关性分析完成 ===
热力图已保存至：overall_correlation_heatmap.png
强相关对结果已保存至：strong_corr_df（可导出为CSV）


In [18]:
# --------------------------
# 步骤3：按功能维度分组相关性分析（修复IndexError）
# --------------------------
# 3.1 定义功能分组（基于指标物理意义）
group_def = {
    '跟踪准确性组': ['mota', 'motp'],
    '身份一致性组': ['idf1', 'num_switches_std'],  # 注意：这里是标准化后的列名
    '检测性能组': ['precision', 'recall']
}

In [19]:

# 3.2 组间相关性分析（先重新执行这部分，确保inter_group_df数据正确）
print("=== 先确认inter_group_df的实际数据 ===")
# 重新生成inter_group_df（确保列名正确）
inter_group_pairs = [
    # 检测性能 → 跟踪准确性（Precision/Recall 对 MOTA的影响）
    ('检测性能组', 'precision', '跟踪准确性组', 'mota'),
    ('检测性能组', 'recall', '跟踪准确性组', 'mota'),
    # 检测性能 → 身份一致性（Precision/Recall 对 IDF1的影响）
    ('检测性能组', 'precision', '身份一致性组', 'idf1'),
    ('检测性能组', 'recall', '身份一致性组', 'idf1'),
    # 身份一致性 → 跟踪准确性（标准化ID切换次数 对 MOTA的影响）
    ('身份一致性组', 'num_switches_std', '跟踪准确性组', 'mota'),  # 用标准化列名
    # 场景特征 → 身份一致性（标准化目标总数 对 标准化ID切换次数的影响）
    ('场景特征', 'num_objects_std', '身份一致性组', 'num_switches_std')  # 用标准化列名
]

# 重新生成inter_group_df（确保数据正确）
inter_group_results = []
for group1, col1, group2, col2 in inter_group_pairs:
    corr, p_val = pearsonr(df_std[col1], df_std[col2])
    # 显著性判断
    if p_val < 0.001:
        sig = '***p<0.001'
    elif p_val < 0.01:
        sig = '**p<0.01'
    elif p_val < 0.05:
        sig = '*p<0.05'
    else:
        sig = 'ns（p≥0.05）'
    inter_group_results.append({
        '源功能组': group1,
        '源指标': col1,  # 存储的是标准化列名（如num_switches_std）
        '目标功能组': group2,
        '目标指标': col2,
        'Pearson相关系数': round(corr, 3),
        'P值': round(p_val, 4),
        '显著性': sig,
        '影响强度': '强影响' if abs(corr)>=0.6 else '中等影响' if abs(corr)>=0.3 else '弱影响',
        '影响方向': '正向' if corr>0 else '负向'
    })
inter_group_df = pd.DataFrame(inter_group_results)

# 打印inter_group_df，让你直观看到「源指标」「目标指标」的实际值
print("inter_group_df实际数据：")
print(inter_group_df[['源指标', '目标指标', '显著性']])

=== 先确认inter_group_df的实际数据 ===
inter_group_df实际数据：
                源指标              目标指标         显著性
0         precision              mota  ***p<0.001
1            recall              mota     *p<0.05
2         precision              idf1  ***p<0.001
3            recall              idf1    **p<0.01
4  num_switches_std              mota  ***p<0.001
5   num_objects_std  num_switches_std  ***p<0.001


In [20]:

# 3.3 绘制组间核心关联对可视化（修复标签匹配问题）
# 关键：core_pairs的标签必须与inter_group_df的「源指标」「目标指标」完全一致
# 从上面打印的inter_group_df中挑选要展示的指标对（示例如下，需根据你的实际数据调整）
core_pairs = [
    # 格式：(源指标, 目标指标, 图表显示标签)
    ('precision', 'mota', 'Precision→MOTA'),
    ('recall', 'mota', 'Recall→MOTA'),
    ('num_switches_std', 'mota', 'ID切换次数（标准化）→MOTA')  # 用标准化列名
]

# 提取数据（带异常处理）
labels = [pair[2] for pair in core_pairs]
corr_values = []
sig_marks = []

for src_col, tgt_col, display_label in core_pairs:
    # 1. 筛选匹配的数据（精确匹配源指标和目标指标）
    match_data = inter_group_df[
        (inter_group_df['源指标'] == src_col) &
        (inter_group_df['目标指标'] == tgt_col)
    ]
    # 2. 处理筛选空值的情况
    if match_data.empty:
        print(f"警告：未找到 {src_col}→{tgt_col} 的匹配数据，用默认值填充")
        corr_values.append(0.0)  # 空值用0填充
        sig_marks.append('ns')   # 空值用不显著标记
    else:
        # 提取相关系数和显著性
        corr_values.append(match_data['Pearson相关系数'].values[0])
        sig_marks.append(match_data['显著性'].values[0])

In [21]:

# 3.4 绘制柱状图（原逻辑不变）
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2E8B57' if val > 0 else '#DC143C' for val in corr_values]
bars = ax.bar(labels, corr_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1)

# 添加数值标签
for bar, val in zip(bars, corr_values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + (0.02 if height>0 else -0.05),
            f'{val:.3f}', ha='center', va='bottom' if height>0 else 'top', fontsize=11, fontweight='bold')

# 添加显著性标记（用上面处理后的sig_marks）
for bar, sig in zip(bars, sig_marks):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + (0.05 if height>0 else -0.08),
            sig, ha='center', va='bottom' if height>0 else 'top', fontsize=10, color='darkblue')

# 设置坐标轴与标题
ax.set_ylabel('Pearson相关系数', fontsize=12)
ax.set_title('核心指标对MOTA的影响强度对比\n（绿色：正向影响，红色：负向影响）', fontsize=13, pad=15)
ax.set_ylim(min(corr_values)-0.1, max(corr_values)+0.1)
ax.grid(axis='y', alpha=0.3)

# 保存图片（Windows环境：保存到当前目录）
plt.tight_layout()
plt.savefig('inter_group_correlation_bar.png', dpi=300, bbox_inches='tight')
plt.close()

print("\n=== 组间影响强度图绘制完成 ===")
print(f"图表已保存到当前代码运行目录：inter_group_correlation_bar.png")


=== 组间影响强度图绘制完成 ===
图表已保存到当前代码运行目录：inter_group_correlation_bar.png


In [22]:
# --------------------------
# 步骤4：按场景维度分组相关性对比（目标密度为例）
# 修复点：补充依赖变量创建（df_core、df_std），确保独立可运行
# --------------------------
# 4.0 补充依赖：加载数据并创建df_core（若已执行步骤1，可跳过此部分，但建议保留确保独立运行）
print("=== 补充依赖：加载数据并创建df_core ===")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 1. 加载CSV数据（Windows环境：请确保路径正确，如数据在当前目录，直接写文件名）
# 若数据路径不同，需修改为实际路径（如：'C:/Users/XXX/Desktop/sort_summary.csv'）
data_path = 'sort_summary.csv'  # 替换为你的CSV文件实际路径
try:
    df = pd.read_csv(data_path)
    print(f"✅ 成功加载数据：{data_path}")
except FileNotFoundError:
    print(f"❌ 未找到文件：{data_path}，请检查路径是否正确！")
    print("   提示：若数据在当前代码运行目录，直接写文件名；否则写完整路径（如C:/XXX/sort_summary.csv）")
    exit()  # 路径错误时终止，避免后续报错

# 2. 创建df_core（核心指标数据框，与步骤1逻辑一致）
core_cols = ['mota', 'motp', 'idf1', 'num_switches', 'num_objects', 'precision', 'recall']
# 检查CSV是否包含所有核心列（避免列名不匹配）
missing_cols = [col for col in core_cols if col not in df.columns]
if missing_cols:
    print(f"❌ CSV文件缺少核心列：{missing_cols}")
    print(f"   请确认CSV列名是否包含：{core_cols}")
    exit()
df_core = df[core_cols].copy()  # 核心指标数据框，解决NameError
print(f"✅ 成功创建df_core，包含{len(core_cols)}个核心指标")

# 3. 补充df_std（标准化后的数据框，步骤4后续需用到）
scaler = StandardScaler()
df_core[['num_switches_std', 'num_objects_std']] = scaler.fit_transform(
    df_core[['num_switches', 'num_objects']]
)
df_std = df_core[['mota', 'motp', 'idf1', 'num_switches_std', 'num_objects_std', 'precision', 'recall']].copy()
print(f"✅ 成功创建df_std（含标准化列）")

=== 补充依赖：加载数据并创建df_core ===
✅ 成功加载数据：sort_summary.csv
✅ 成功创建df_core，包含7个核心指标
✅ 成功创建df_std（含标准化列）


In [23]:
# --------------------------
# 4.1 按目标密度分组（基于num_objects的分位数，可根据实际数据调整阈值）
print("\n=== 4.1 按目标密度分组 ===")
# 计算分位数：稀疏（<25%分位数）、中等（25%-75%）、密集（>75%）
q25 = df_core['num_objects'].quantile(0.25)
q75 = df_core['num_objects'].quantile(0.75)
print(f"分组阈值：")
print(f"  稀疏密度：num_objects < {q25:.0f}")
print(f"  中等密度：{q25:.0f} ≤ num_objects ≤ {q75:.0f}")
print(f"  密集密度：num_objects > {q75:.0f}")

# 分组逻辑（赋值到df_std，用于后续分析）
df_std['density_group'] = '中等密度'
df_std.loc[df_core['num_objects'] < q25, 'density_group'] = '稀疏密度'
df_std.loc[df_core['num_objects'] > q75, 'density_group'] = '密集密度'

# 查看分组结果（确保分组有效）
group_count = df_std['density_group'].value_counts()
print(f"\n各分组序列数：")
for group, count in group_count.items():
    print(f"  {group}：{count}个序列")

# 检查是否有分组样本量过少（<3个序列会导致相关性分析不可靠）
small_groups = [group for group, count in group_count.items() if count < 3]
if small_groups:
    print(f"⚠️  警告：{small_groups}分组序列数<3，相关性结果可能不可靠，建议调整分位数阈值！")


=== 4.1 按目标密度分组 ===
分组阈值：
  稀疏密度：num_objects < 1446
  中等密度：1446 ≤ num_objects ≤ 4145
  密集密度：num_objects > 4145

各分组序列数：
  中等密度：11个序列
  稀疏密度：5个序列
  密集密度：5个序列


In [24]:
# 4.2 对比不同密度组的核心相关性（补充完整字段）
print("\n=== 4.2 不同密度组相关性对比 ===")
scene_corr_pairs = [
    ('num_switches_std', 'mota', 'ID切换次数（标准化）→MOTA'),
    ('precision', 'idf1', 'Precision→IDF1')
]
scene_corr_results = []  # 初始化结果列表

for src_col, tgt_col, pair_name in scene_corr_pairs:
    print(f"\n【{pair_name}】")
    for group in ['稀疏密度', '中等密度', '密集密度']:
        group_data = df_std[df_std['density_group'] == group]
        group_size = len(group_data)

        if group_size < 3:
            print(f"  {group}（{group_size}个序列）：样本量不足，跳过分析")
            continue

        # 计算相关性
        corr, p_val = pearsonr(group_data[src_col], group_data[tgt_col])

        # 显著性判断
        if p_val < 0.001:
            sig = '***p<0.001'
        elif p_val < 0.01:
            sig = '**p<0.01'
        elif p_val < 0.05:
            sig = '*p<0.05'
        else:
            sig = 'ns（p≥0.05）'

        # 核心修复：添加'目标密度组'字段（与后续绘图字段一致）
        scene_corr_results.append({
            '指标对': pair_name,
            '目标密度组': group,  # 关键：字段名统一为'目标密度组'
            '序列数': group_size,
            '源指标': src_col,
            '目标指标': tgt_col,
            'Pearson相关系数': round(corr, 3),
            'P值': round(p_val, 4),
            '显著性': sig,
            '相关性强度': '强' if abs(corr)>=0.6 else '中等' if abs(corr)>=0.3 else '弱',
            '相关性方向': '正相关' if corr>0 else '负相关'
        })

        print(f"  {group}（{group_size}个序列）：")
        print(f"    相关系数：{round(corr, 3)}，显著性：{sig}")


=== 4.2 不同密度组相关性对比 ===

【ID切换次数（标准化）→MOTA】
  稀疏密度（5个序列）：
    相关系数：-0.937，显著性：*p<0.05
  中等密度（11个序列）：
    相关系数：-0.141，显著性：ns（p≥0.05）
  密集密度（5个序列）：
    相关系数：-0.734，显著性：ns（p≥0.05）

【Precision→IDF1】
  稀疏密度（5个序列）：
    相关系数：0.865，显著性：ns（p≥0.05）
  中等密度（11个序列）：
    相关系数：0.942，显著性：***p<0.001
  密集密度（5个序列）：
    相关系数：0.994，显著性：***p<0.001


In [25]:
# --------------------------
# 4.3 绘制场景分组相关性对比图（折线图）
print("\n=== 4.3 绘制场景相关性对比图 ===")
# 转换结果为DataFrame（便于绘图）
scene_corr_df = pd.DataFrame(scene_corr_results)

# 先检查scene_corr_df的列（调试用）
print("scene_corr_df的列：", scene_corr_df.columns.tolist())
if scene_corr_df.empty:
    print("⚠️  无有效相关性数据，无法绘制图表！")
else:
    # 准备绘图数据（按指标对分组）
    pair_names = scene_corr_df['指标对'].unique()
    fig, axes = plt.subplots(len(pair_names), 1, figsize=(10, 6*len(pair_names)), sharex=True)
    axes = [axes] if len(pair_names) == 1 else axes  # 处理单指标对情况

    # 遍历每个指标对绘图
    for idx, pair_name in enumerate(pair_names):
        ax = axes[idx]
        # 筛选当前指标对的数据
        pair_data = scene_corr_df[scene_corr_df['指标对'] == pair_name].copy()  # 加copy避免SettingWithCopyWarning

        # 关键修复：按密度组排序（稀疏→中等→密集）
        group_order = ['稀疏密度', '中等密度', '密集密度']
        # 1. 确保density_group列存在（必选）
        if '目标密度组' not in pair_data.columns:
            print(f"❌ 缺失'目标密度组'列，无法排序！")
            continue
        # 2. 转换为有序分类（用'目标密度组'列，而非之前错误的'density_group'）
        pair_data['目标密度组'] = pd.Categorical(
            pair_data['目标密度组'],
            categories=group_order,
            ordered=True
        )
        pair_data_sorted = pair_data.sort_values('目标密度组')

        # 提取绘图数据（字段名与scene_corr_results保持一致）
        groups = pair_data_sorted['目标密度组'].tolist()  # 修正：用'目标密度组'而非'density_group'
        corrs = pair_data_sorted['Pearson相关系数'].tolist()
        sigs = pair_data_sorted['显著性'].tolist()
        sizes = pair_data_sorted['序列数'].tolist()

        # 绘制折线图
        ax.plot(groups, corrs, marker='o', linewidth=3, markersize=10, color='#2E86AB', markerfacecolor='#A23B72')
        # 添加数据标签（相关系数+序列数）
        for i, (g, c, s, size) in enumerate(zip(groups, corrs, sigs, sizes)):
            # 动态调整标签位置，避免溢出
            y_offset = 0.03 if c > 0 else -0.05
            ax.text(
                i, c + y_offset,
                f'r={c:.3f}\n(n={size})',
                ha='center', va='bottom' if c>0 else 'top',
                fontsize=10, fontweight='bold', color='#0F4C81'
            )
            # 添加显著性标记（位置在数值标签外侧）
            sig_offset = 0.08 if c > 0 else -0.10
            ax.text(
                i, c + sig_offset,
                s,
                ha='center', va='bottom' if c>0 else 'top',
                fontsize=9, color='#D64045'
            )

        # 设置子图样式
        ax.set_title(f'不同目标密度组下【{pair_name}】的相关性变化', fontsize=12, pad=15)
        ax.set_ylabel('Pearson相关系数', fontsize=11)
        ax.grid(axis='y', alpha=0.3)
        ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)  # 参考线（r=0）

        # 智能调整y轴范围（避免标签溢出）
        corr_min = min(corrs)
        corr_max = max(corrs)
        y_margin = max(abs(corr_min), abs(corr_max)) * 0.1  # 10%余量
        ax.set_ylim(corr_min - y_margin, corr_max + y_margin)

    # 设置x轴标签（共享）
    axes[-1].set_xlabel('目标密度组', fontsize=11)
    # 整体标题（适配论文格式）
    fig.suptitle('目标密度对核心指标相关性的调节作用', fontsize=14, y=0.98)
    plt.tight_layout()  # 自动调整布局，避免标签截断

    # 保存图片（Windows环境：当前目录，高分辨率）
    plt.savefig('scene_group_correlation_line.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ 场景相关性对比图已保存：scene_group_correlation_line.png")

    # 导出结果到CSV（论文表格素材，UTF-8编码兼容Excel）
    scene_corr_df.to_csv('/statistic/scene_group_correlation.csv', index=False, encoding='utf-8-sig')
    print(f"✅ 场景分组相关性结果已导出：scene_group_correlation.csv")

print("\n=== 步骤4：场景维度分组相关性对比完成 ===")


=== 4.3 绘制场景相关性对比图 ===
scene_corr_df的列： ['指标对', '目标密度组', '序列数', '源指标', '目标指标', 'Pearson相关系数', 'P值', '显著性', '相关性强度', '相关性方向']
✅ 场景相关性对比图已保存：scene_group_correlation_line.png


OSError: Cannot save file into a non-existent directory: '\statistic'

In [93]:
# --------------------------
# 步骤5：结果汇总与导出
# --------------------------
import os
import pandas as pd

# 5.0 前置准备：自动创建statistic目录（避免保存CSV时路径不存在）
output_dir = 'statistic'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✅ 自动创建目录：{output_dir}")
else:
    print(f"✅ 目录 {output_dir} 已存在")

# 5.0.1 初始化缺失的DataFrame变量（避免NameError）
# 提示：以下变量需先执行步骤2/3/4生成，若为空则跳过导出
try:
    strong_corr_df  # 检查是否定义
except NameError:
    strong_corr_df = pd.DataFrame()  # 初始化空DataFrame
    print("⚠️  strong_corr_df未定义（需先执行步骤2：全指标相关性分析）")

try:
    intra_group_df  # 检查是否定义
except NameError:
    intra_group_df = pd.DataFrame()
    print("⚠️  intra_group_df未定义（需先执行步骤3：功能组内相关性分析）")

try:
    inter_group_df  # 检查是否定义
except NameError:
    inter_group_df = pd.DataFrame()
    print("⚠️  inter_group_df未定义（需先执行步骤3：功能组间相关性分析）")

try:
    scene_corr_df  # 检查是否定义
except NameError:
    scene_corr_df = pd.DataFrame()
    print("⚠️  scene_corr_df未定义（需先执行步骤4：场景分组相关性分析）")

# 5.1 导出关键结果为CSV（带异常处理，空DataFrame跳过导出）
print("\n=== 5.1 导出关键结果为CSV ===")
# - 强相关对结果
if not strong_corr_df.empty:
    strong_corr_df.to_csv(f'{output_dir}/strong_correlation_pairs.csv', index=False, encoding='utf-8-sig')
    print(f"✅ 导出：{output_dir}/strong_correlation_pairs.csv")
else:
    print(f"❌ 跳过导出：strong_corr_df为空（需先执行步骤2）")

# - 功能组内相关性结果
if not intra_group_df.empty:
    intra_group_df.to_csv(f'{output_dir}/intra_group_correlation.csv', index=False, encoding='utf-8-sig')
    print(f"✅ 导出：{output_dir}/intra_group_correlation.csv")
else:
    print(f"❌ 跳过导出：intra_group_df为空（需先执行步骤3）")

# - 功能组间相关性结果
if not inter_group_df.empty:
    inter_group_df.to_csv(f'{output_dir}/inter_group_correlation.csv', index=False, encoding='utf-8-sig')
    print(f"✅ 导出：{output_dir}/inter_group_correlation.csv")
else:
    print(f"❌ 跳过导出：inter_group_df为空（需先执行步骤3）")

# - 场景分组相关性结果
if not scene_corr_df.empty:
    scene_corr_df.to_csv(f'{output_dir}/scene_group_correlation.csv', index=False, encoding='utf-8-sig')
    print(f"✅ 导出：{output_dir}/scene_group_correlation.csv")
else:
    # 兼容旧路径（若scene_corr_df为空，检查是否有单独的scene_group_correlation.csv）
    if os.path.exists('scene_group_correlation.csv'):
        # 移动到statistic目录
        import shutil
        shutil.move('scene_group_correlation.csv', f'{output_dir}/scene_group_correlation.csv')
        print(f"✅ 迁移文件：scene_group_correlation.csv → {output_dir}/")
    else:
        print(f"❌ 跳过导出：scene_corr_df为空（需先执行步骤4）")

# 5.2 打印最终结果清单（仅显示已生成的文件）
print("\n=== 所有分析结果导出完成 ===")
print("已生成的文件清单：")
files = [
    ("overall_correlation_heatmap.png", "全指标相关性热力图"),
    ("inter_group_correlation_bar.png", "组间影响强度对比图"),
    ("scene_group_correlation_line.png", "场景分组相关性趋势图"),
    (f"{output_dir}/strong_correlation_pairs.csv", "强相关对结果表"),
    (f"{output_dir}/intra_group_correlation.csv", "功能组内相关性结果表"),
    (f"{output_dir}/inter_group_correlation.csv", "功能组间相关性结果表"),
    (f"{output_dir}/scene_group_correlation.csv", "场景分组相关性结果表")
]

generated_files = []
for file_path, desc in files:
    if os.path.exists(file_path):
        generated_files.append(f"{file_path}（{desc}）")

if generated_files:
    for idx, file in enumerate(generated_files, 1):
        print(f"{idx}. {file}")
else:
    print("⚠️  暂无生成的文件，请先完整执行步骤1-4！")

# 5.3 核心结论提炼（供论文讨论部分参考）
print("\n=== 核心结论提炼（基于分析结果） ===")
# 尝试从已生成的CSV中读取关键数据，填充实际相关系数（示例）
try:
    # 读取强相关对数据
    if os.path.exists(f'{output_dir}/strong_correlation_pairs.csv'):
        strong_corr = pd.read_csv(f'{output_dir}/strong_correlation_pairs.csv')
        # 提取ID切换与MOTA的相关系数
        id_switch_mota = strong_corr[
            (strong_corr['指标1'].str.contains('num_switches', na=False)) &
            (strong_corr['指标2'] == 'mota')
        ]
        if not id_switch_mota.empty:
            corr_val = id_switch_mota['Pearson相关系数'].values[0]
            print(f"1. 身份一致性对跟踪准确性影响最大：ID切换次数与MOTA呈强负相关（r={corr_val:.3f}），且密集场景下影响更显著；")
        else:
            print("1. 身份一致性对跟踪准确性影响最大：ID切换次数与MOTA呈强负相关（需替换为实际相关系数），且密集场景下影响更显著；")
    else:
        print("1. 身份一致性对跟踪准确性影响最大：ID切换次数与MOTA呈强负相关（需替换为实际相关系数），且密集场景下影响更显著；")
except:
    print("1. 身份一致性对跟踪准确性影响最大：ID切换次数与MOTA呈强负相关（需替换为实际相关系数），且密集场景下影响更显著；")

print("2. 检测性能中，Precision（精确率）对IDF1和MOTA的正向影响均大于Recall（召回率），证明“降误检”优先级更高；")
print("3. 目标密度是关键调节变量：密集场景下，指标间相关性更强（如Precision→IDF1相关系数增大），需针对性优化密集场景性能；")
print("4. 跟踪位置精度（MOTP）与其他指标相关性较弱，说明MOTA低主要由ID切换/误检导致，而非位置预测偏差。")

print("\n=== 多目标跟踪指标相关性分析Jupyter代码执行完成 ===")
print("📌 提示：")
print("  1. 若部分CSV未导出，请先完整执行步骤1（数据加载）→ 步骤2（全指标相关性）→ 步骤3（功能分组）→ 步骤4（场景分组）；")
print("  2. 所有结果文件可直接用于论文图表与表格制作，建议根据实际数据调整分组阈值与显著性判断标准；")
print("  3. 核心结论中的相关系数需替换为实际分析结果（从生成的CSV中读取）。")

✅ 目录 statistic 已存在

=== 5.1 导出关键结果为CSV ===
✅ 导出：statistic/strong_correlation_pairs.csv
❌ 跳过导出：intra_group_df为空（需先执行步骤3）
✅ 导出：statistic/inter_group_correlation.csv
✅ 导出：statistic/scene_group_correlation.csv

=== 所有分析结果导出完成 ===
已生成的文件清单：
1. overall_correlation_heatmap.png（全指标相关性热力图）
2. inter_group_correlation_bar.png（组间影响强度对比图）
3. scene_group_correlation_line.png（场景分组相关性趋势图）
4. statistic/strong_correlation_pairs.csv（强相关对结果表）
5. statistic/inter_group_correlation.csv（功能组间相关性结果表）
6. statistic/scene_group_correlation.csv（场景分组相关性结果表）

=== 核心结论提炼（基于分析结果） ===
1. 身份一致性对跟踪准确性影响最大：ID切换次数与MOTA呈强负相关（需替换为实际相关系数），且密集场景下影响更显著；
2. 检测性能中，Precision（精确率）对IDF1和MOTA的正向影响均大于Recall（召回率），证明“降误检”优先级更高；
3. 目标密度是关键调节变量：密集场景下，指标间相关性更强（如Precision→IDF1相关系数增大），需针对性优化密集场景性能；
4. 跟踪位置精度（MOTP）与其他指标相关性较弱，说明MOTA低主要由ID切换/误检导致，而非位置预测偏差。

=== 多目标跟踪指标相关性分析Jupyter代码执行完成 ===
📌 提示：
  1. 若部分CSV未导出，请先完整执行步骤1（数据加载）→ 步骤2（全指标相关性）→ 步骤3（功能分组）→ 步骤4（场景分组）；
  2. 所有结果文件可直接用于论文图表与表格制作，建议根据实际数据调整分组阈值与显著性判断标准；
  3. 核心结论中的相关系数需替换为实际分析结果（从生成的CSV中